# PSET 1: Bottom-Up Synthesis

I follow Algorithm 1 in the BUSTLE paper:

> Odena, A. *et al.* BUSTLE: Bottom-Up Program Synthesis Through Learning-Guided Exploration. in *9th International Conference on Learning Representations*; 2021 May 3-7; Austria.

First, I import the required libraries.

In [1]:
import numpy as np
import itertools

# argument parser for command line arguments
import argparse

# import arithmetic module
from arithmetic import *
from abstract_syntax_tree import OperatorNode
from examples import example_set, check_examples
from synthesizer import extract_constants, observationally_equivalent, check_program
import config

First, I define variables as proxies for command-line arguments provided to the synthesizer.

In [2]:
domain = "arithmetic"
examples_key = "addition"
examples = example_set[examples_key]
max_weight = 3

I define a function to extract constants from examples.

In [3]:
# initialize program bank
program_bank = extract_constants(examples)
program_bank_str = [p.str() for p in program_bank]

Next, I define the bottom-up synthesis algorithm.

In [4]:
# define operators
operators = arithmetic_operators

# iterate over each level
for weight in range(2, max_weight):

    for op in operators:

        # get all possible combinations of primitives in program bank
        combinations = itertools.combinations(program_bank, op.arity)

        # iterate over each combination
        for combination in combinations:

            # get type signature
            type_signature = [p.type for p in combination]

            # check if type signature matches operator
            if type_signature != op.arg_types:
                continue

            # check that sum of weights of arguments <= w
            if sum([p.weight for p in combination]) > weight:
                continue

            # create new program
            program = OperatorNode(op, combination)

            # check if program is in program bank using string representation
            if program.str() in program_bank_str:
                continue
            
            # check if program is observationally equivalent to any program in program bank
            if any([observationally_equivalent(program, p, examples) for p in program_bank]):
                continue

            # add program to program bank
            program_bank.append(program)
            program_bank_str.append(program.str())

            # check if program passes all examples
            if check_program(program, examples):
                # return(program)
                print(program.str())

(x0 + x1)


"<class 'int'>"